<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Text-summarization" data-toc-modified-id="Text-summarization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Text summarization</a></span><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Анализ-всевозможных-алгоритмов-саммаризации" data-toc-modified-id="Анализ-всевозможных-алгоритмов-саммаризации-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Анализ всевозможных алгоритмов саммаризации</a></span></li><li><span><a href="#Описание-тестового-набора-данных" data-toc-modified-id="Описание-тестового-набора-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Описание тестового набора данных</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Similarity-metrics-(если-понадобится)" data-toc-modified-id="Similarity-metrics-(если-понадобится)-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Similarity metrics (если понадобится)</a></span></li><li><span><a href="#Embeddings-(если-понадобиться-эксперементировать-с-вариациями-embedding)" data-toc-modified-id="Embeddings-(если-понадобиться-эксперементировать-с-вариациями-embedding)-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Embeddings (если понадобиться эксперементировать с вариациями embedding)</a></span></li><li><span><a href="#Метрики" data-toc-modified-id="Метрики-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Метрики</a></span></li><li><span><a href="#Extractive-methods" data-toc-modified-id="Extractive-methods-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Extractive methods</a></span><ul class="toc-item"><li><span><a href="#Word-frequency-method" data-toc-modified-id="Word-frequency-method-1.8.1"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>Word frequency method</a></span></li><li><span><a href="#TextRank" data-toc-modified-id="TextRank-1.8.2"><span class="toc-item-num">1.8.2&nbsp;&nbsp;</span>TextRank</a></span></li><li><span><a href="#ClusterRank" data-toc-modified-id="ClusterRank-1.8.3"><span class="toc-item-num">1.8.3&nbsp;&nbsp;</span>ClusterRank</a></span></li><li><span><a href="#LSA" data-toc-modified-id="LSA-1.8.4"><span class="toc-item-num">1.8.4&nbsp;&nbsp;</span>LSA</a></span></li><li><span><a href="#Вручную-создаем-матрицу-частот-td-idf" data-toc-modified-id="Вручную-создаем-матрицу-частот-td-idf-1.8.5"><span class="toc-item-num">1.8.5&nbsp;&nbsp;</span>Вручную создаем матрицу частот td-idf</a></span></li><li><span><a href="#Созданием-матрицы-td-idf-через-библиотеку" data-toc-modified-id="Созданием-матрицы-td-idf-через-библиотеку-1.8.6"><span class="toc-item-num">1.8.6&nbsp;&nbsp;</span>Созданием матрицы td-idf через библиотеку</a></span></li><li><span><a href="#KLsum-(нужно-немного-улучшить-содеражние-саммари,-поставить-ограничение-на-количество-предложений)" data-toc-modified-id="KLsum-(нужно-немного-улучшить-содеражние-саммари,-поставить-ограничение-на-количество-предложений)-1.8.7"><span class="toc-item-num">1.8.7&nbsp;&nbsp;</span>KLsum (нужно немного улучшить содеражние саммари, поставить ограничение на количество предложений)</a></span></li><li><span><a href="#Mead" data-toc-modified-id="Mead-1.8.8"><span class="toc-item-num">1.8.8&nbsp;&nbsp;</span>Mead</a></span></li></ul></li><li><span><a href="#Abstractive-methods" data-toc-modified-id="Abstractive-methods-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Abstractive methods</a></span><ul class="toc-item"><li><span><a href="#Bert" data-toc-modified-id="Bert-1.9.1"><span class="toc-item-num">1.9.1&nbsp;&nbsp;</span>Bert</a></span></li><li><span><a href="#GPT2" data-toc-modified-id="GPT2-1.9.2"><span class="toc-item-num">1.9.2&nbsp;&nbsp;</span>GPT2</a></span></li><li><span><a href="#XLNet" data-toc-modified-id="XLNet-1.9.3"><span class="toc-item-num">1.9.3&nbsp;&nbsp;</span>XLNet</a></span></li><li><span><a href="#Hugging-face-transformer-(BERT)" data-toc-modified-id="Hugging-face-transformer-(BERT)-1.9.4"><span class="toc-item-num">1.9.4&nbsp;&nbsp;</span>Hugging face transformer (BERT)</a></span></li></ul></li><li><span><a href="#Идеи-для-чего-то-нового" data-toc-modified-id="Идеи-для-чего-то-нового-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Идеи для чего-то нового</a></span></li><li><span><a href="#Проблемы,-с-которыми-можно-будет-побороться" data-toc-modified-id="Проблемы,-с-которыми-можно-будет-побороться-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Проблемы, с которыми можно будет побороться</a></span></li><li><span><a href="#Распознование-речи" data-toc-modified-id="Распознование-речи-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Распознование речи</a></span></li><li><span><a href="#Список-литературы" data-toc-modified-id="Список-литературы-1.13"><span class="toc-item-num">1.13&nbsp;&nbsp;</span>Список литературы</a></span></li></ul></li></ul></div>

# Text summarization

## Введение

*+* общая формулировка темы;

*+* актуальность выбранной темы, её теоретическое и/или практическое значение;

*+* степень научной проработанности темы исследования;

*+* цель и задачи исследования;

*+* объяснение того, как и в каком порядке, автор намеревается решать поставленные задачи;

*+* анонс структуры работы (названия глав работы и их краткая характеристика);

*+* характеристика основных источников информации.

Введение должно быть кратким (1-3 страницы) и четким. Из введения должно быть понятно, чему посвящена работа (цель работы), какие задачи и с помощью каких методов в ней решаются, какие результаты должны быть получены в ходе выполнения курсовой работы.

**Исследование саммаризации текстов в применении к реферированию переговоров**

**Общая формулировка темы**:  
Задача саммаризации в применении к реферированию деловых встреч заключается в выделении конкретного подмножества  информации, которое отражает основную суть исходного документа, в нашем случае аудиозаписи. 

**Актуальность выбранной темы:**   
Устойчивые тенденции последнего десятилетия показывают высокую интегрированность NLP в различных сферах деятельности. Список задач, решаемых NLP, постоянно расширяется, на данный момент он включает в себя: анализ текста, генерирование текста, машинный перевод, определение частей речи слов, категоризация текста, автоматическое создание субтитров и другое.  В крупнейших компаниях, таких, как Google, Microsoft, Amazon, IBM NLP-подразделения включают в себя все больше  и больше сотрудников. С каждым годом появляются все новые модификации алгоритмов работы с естественным языком, например, GPT-3 и все больше они внедряются в повседневную жизнь, например, активно начали использоваться голосовые помощники. 
Задача саммаризации, также называемая задачей реферирования, становится наиболее популярной в настоящее время особенно с ростом количества информации. Со временем анализ текстовой информации вручную стал рутинной работой, которую возможно автоматизировать. Каждый день мы работаем с текстовыми данными, оставляя отзыв в различных организациях, описывая свои проблемы или отправляя запрос в Интернет. Но чем больше данных в пространстве Интернета, тем меньше времени на ознакомление с ними, все сложнее найти релевантную информацию за оптимальное количество времени.  Таким образом, появляется необходимость в отбрасывании несущественной и в выделении уникальной информации из прочитанного, увиденного или услышанного. До этого момента задача саммаризации применялась в основном в новостном канале или для категоризации отзывов или статей на сайтах с использованием англоязычных наборов данных, в данной работе мы попробуем применить модели к русскоязычным наборам данных для достижения бизнес-целей. 

**Теоретическое значение**:  
В данной работе комбинируются алгоритмы двух различных направлений: распознавания речи и естественного языка, тем самым раскрывается все больший потенциал развития NLP в других сферах деятельности.  Результаты послужат толчком к развитию обработки естественного языка при помощи русскоязычных наборов данных и откроется новый взгляд на вопрос о том, какие метрики и алгоритмы наиболее подходящие для данных о деловых встречах. 

**Практическое значение**:   
За крупными компаниями прослеживаются тенденции в горизонтальном расширении своих продуктов в различных сферах. С ростом количества направлений, многим отделам все сложнее интегрироваться в работу других отделов и чувствовать себя единой системой. За день в компаниях проводится достаточно большое количество встреч, генерируются все больше новых предложений и идей, в результате чего достаточно трудозатратно успевать за полетом мысли руководителей разных отделений. К тому же, у сотрудников недостаточно времени, чтобы присутствовать на всех мероприятиях, организованных компанией, поэтому в данной сфере практическое применение находит саммаризация огромного количества информации для общей осведомленности сотрудников об актуальных новостях и идеях. 

**Степень научной проработанности темы исследования**:  
Саммаризация в NLP является достаточно новым, постоянно развивающемся явлением.  Только в  1958 году появились алгоритмы, извлекающие самое важное из текстовых данных, такие как Luhn. Они были основаны на частотном подходе, который не учитывал ни контекст предложений, ни  связь предложения с другими. Первая метрика,  способная оценить качество данных моделей была придумана советским математиком в 1965 году, она называлась “Расстояние Левенштейна”, но она рассчитывалась по-символьно: сколько замен в символах нужно произвести, чтобы одна последовательность была максимально схожа с другой. Наиболее релевантная метрика BLEU (BiLingual Evaluation Understudy), учитывающая не символы последовательности, а различных комбинации слов (n-граммы) была придумана в 2002 году одной из команд  IBM. Основные открытия в языковых моделях была сделаны в XXI веке: Google разработала алгоритм BERT в 2019 году, основная идея которого заключается в способности анализировать запрос, как цельное предложение, основываясь на контексте; GPT-3 от OpenAI - самая продвинутая языковая модель в мире, умеющая решать любые задачи, сформулированные на английском языке, разработана в 2020 году. Таким образом, область языковых моделей является постоянно развивающейся, но в применении к деловым совещаниям пока что языковые модели не применялись, поэтому можем считать эту тему исследования достаточно актуальной. 

**Цель исследования:**  
- Разработать модели, применимые для русскоязычного набора данных  
- Выявить связь между распознаванием речи и реферированием текста  
- Подобрать метрики, которые наиболее точно оценивают эффективность применяемых моделей   
- Выявить преимущества и недостатки рассмотренных моделей  

**Поставленные задачи в порядке их выполнения:**  
- Исследовать данную сферу деятельности на степень научной проработанности  
- Определиться с тестовым набором данных  
- Изучить классификацию алгоритмов саммаризации  
- Изучить способы оценки качества алгоритмов реферирования    
- Реализовать работу алгоритмов и оценить качество их работы    
- Провести сравнительный анализ, используя различные метрики    
- Изучить алгоритмы распознавания речи  
- Подумать над модификациями алгоритмов саммаризации   
- Скомбинировать алгоритмы распознавания речи и саммаризации   

Для начала нужно аккуратно подойти к выбору данных, с которыми придется работать. Данные должны иметь максимальную тематическую схожесть с рассматриваемой областью. Имея данные с саммари для каждого из текстовых документов можно приступать к изучению классификации алгоритомов саммаризации. Проведя анализ преимуществ и недостатков алгоритмов, можно перейти к способам оценивания качества, после чего уже применить алгоритмы на данных и провести сравнительный анализ, сопоставляя различные метрики и алгоритмы. На данном этапе можно приступать к исследованию алгоритмов распознования речи в применении к задаче саммаризации или независимо от нее, если таковых источников не имеется. Аналогично провести сравнительный анализ, анализируя преимущества и недостатки различных библиотек для распознавания речи. Перед тем как комбинировать алгоритмы распознавания речи и реферирования текстов, нужно снова определиться с данными, с которыми мы будем работать. В этот раз в наличии должы быть аудиофайлы и их краткое содержание. Определившись с датасетом, снова смотрим на метрики и по ним выбираем итоговую модель, которая показывает наилучшие результаты по сравнению с другими. 

**Обзор литературы (характеристика основных источников информации):** 

Первая статья на тему частотного метода саммаризации Luhn была опубликована в 1958 году. С 1960-х годов было разработано огромное количество метододов, которые в последтсвии уже стали классифицировать. 
Можем выделить основных два направления входных данных: [2]
1. Реферирование одиночных документов (на вход подается один текстовый документ)   
2. Реферирование нескольких документов (на вход подается несколько документов, например, связанные одной тематикой и нужно извлечь основную информацию по данной тематике) 

А также можно было выделить три способа саммаризации: 
1. Extractive summarization. [6]  
Данный алгоритм можно сравнить с маркером для выявления основной информации. Исследуем текст, находим  значимые по весу предложения с помощью различных алгоритмов и извлекаем наиболее весомые предложения. У  такого подхода есть недостатки, основной из них - отсутствие баланса и плавности в переходе от одного предложения к другому, но основное преимущество заключается в простоте использования данного подхода, в процессе анализа текста не приходится задумываться о грамматике, семантической связи между словами и предложениями.  
2. Abstractive summarization [7]  
Данный подход уже больше похож на то, как работает интеллект людей при виде огромного набора текстовых данных : то есть считываем текст, перефразируем его, извлекая основную суть, с помощью слов, которых нет в основном документе. 
3. Hybrid  (смесь двух вышеупомянутых подходов) 

Классификация методов саммаризации: 
- Вероятностные и частотные подходы [11]  [9] [1]  
Вероятностные подходы используют статистические характеристики текста для определение веса предложения или строят распределение саммари такое, что оно схоже с распределением исходного текста, но содержит минимальное количество слов.  
- Подходы, основанные на графах [5]  
В качестве вершин графа используются предложения, а связи между преддожения - взвешенные ребра. С помощью различных алгоритмов каждой вершине присваиваются определнные веса, а затем отбираются предложения с наибольшими весами.  
- Нейронные сети [8]  
Нейронные сети для построения реферирования текста основываются на структуре encoder-decoder совместно с LSTM.  
- Подходы, использующие машинное обучение  
Для выявления наиболее оптимального диверсифицированного содеражния исходного документа применяют алгоритмы кластеризации, которые в качестве саммари берут все центроиды, обнаруженные алгоритмом.  
- Тематическое моделирование [12].  
Тематическое моделирование, например, LDA или LSA, заключается в выделении основных топиков текста, а затем вокруг для каждого топика выделяется 1 - 2 предложения, которые впоследтсвии войдут в саммари.  

## Анализ всевозможных алгоритмов саммаризации  

Итак, начнем с самого начала истории саммаризации. 
1. ***Positional method in 1958 by P.Baxendale***  [9]   
В 85% случаев первое предложение было смысловым, а в 7% случаев последнее предложение было смысловым, то есть определяющее основную суть текста, поэтому решили брать первое и последнее предложение текстового документа в итоговый реферат.  
2. ***Luhn's method H.P.Luhn in 1958***  [1]  
Анализировал, как часто слова появляются в документе. Убираем самые частые слова и редкие слова. Затем делаем дата-клининг посредством стемминга и извлечением стоп-слов. Дальше уже считаем частоту для каждого предложения. Например, у нас есть предложение из 10 слов, там 4 основных слова, как мы определяем, что предложение главное? Мы берем 4^2 и делим на колчисетво слов между первым важным и последним важным, допустим у нас их будет 6, поэтому считаем, что вес нашего предложения: 16/6  Потом в реферат мы берем предложения с наибольшим весом, количество таких предложений задается в качестве параметра. 
3. ***H.P.EDMUNSON 1968***  [3]  
Edmunson также обращал внимание на позицию предложения (P) в тексте и на частоту слов (F), как в предыдущих двух способах, но он добавил три типа ключевых слов (C): bonus words - показывают важность предложения, stigma words - негативно влияют на важность предложения (алгоритм считает их важными, но самом деле они не важные), null words (нейтральные для важности). Смотрел на структуру документа (S), находится ли это предложение в названии, под названием, в главном параграфе и тд. А затем в качестве оценки для каждого предложения он предложил использовать линейную комбинацию: a1 * P + a2 * F + a3 * C + a4 * S   
4. ***FRUMP (Fast reading understanding and memorable program) 1979 introduced by G.deJong***  [4]  
Совсем другой подход к данной проблеме, G.deJong хотел применить саммаризацию к новостям и создал 50 sketchy scripts, которые описывают типичные ситуация в новостях, а потом когда мы получаем новый текст для реферирования, то мы заполняем пропуски в этих скриптах наиболее подходящими словами из текста, но это оказался нерабочий подход, 50 сценариев было недостаточно для всех мировых проблем.
5. ***Classification Kupiec 1995***  
Суть подхода заключалась в классификации. Kupiec вручную выбирал документы и вручную составлял краткие описания по ним для своей тренировочной выборки. Для задачи он использовал наивный байесовский классификатор. Вероятность, что предложение s принадлежит краткому описанию, при том что есть какой-то набор признаков F1, F2, F3... это то же самое, что и вероятность, что эти признаки есть у предложения s в его описании * вероятность, что это предложение в описании / вероятность, что эти признаки появляются вместе.  
$ P(s \in S|F_1, F_2...F_k) = \frac{P(F_1, F_2...F_k|s \in S) * P(s \in S)}{P(F_1, F_2...F_k)}$  
Предполагая статистическую независимость признаков, можем записать так:  
$ P(s \in S|F_1, F_2...F_k) = \frac{\prod\limits_{j = 1}^k P(F_j|s \in S) * P(s \in S)}{\prod\limits_{j = 1}^k P(F_j)}$   
Precision этой модели была достаточно высокой: 84%. Но в 2002 году Miles Osborne доказал, что метод энтропии рыботает эффективнее, потому что признаки скорее всего зависимы друг от друга. 
6. ***Miles Osborne 2002 Maximum entropy for setntence extraction***  
Аналогичный подход, что и в прошлом пункте, но вместе наивного байесовского классификатора, мы используем формулу максимальной энтропии.  
7. ***Частотный подход (TDIDF)***  
TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова ${t_i}$ в пределах отдельного документа.  
$tf(t, d) =\frac{n_t}{\sum_{i=1} n_k} $,  
где ${n_t}$ есть число вхождений слова в t документ, а в знаменателе — общее число слов в данном документе.  
idf(t, D) = $\log \frac{|D|}{|{(d_i \in D|t \in d_i)}|}$  
tf - idf = tf * idf  
Выбор основания логарифма в формуле не имеет значения, поскольку изменение основания приводит к изменению веса каждого слова на постоянный множитель, что не влияет на соотношение весов.  
|D| - число документов в коллекции;  
${|(d_i \in D|t \in d_i)|}$ - число документов из коллекции D, в которых встречается t когда ${n_i}$ !=0  
Применительно к нашему случаю, скорее всего документ - предложение. То есть, для понимания приведем пример: 
Если наше предложение содержит 25 слов, и слово «заяц» встречается в нём 3 раза, то частота слова (TF) для слова «заяц» в документе будет 0,12 (3/25). Вычислим IDF как десятичный логарифм отношения количества всех документов к количеству предложений содержащих слово «заяц». Таким образом, если «заяц» содержится в 1000 предложениях из 10 000 000 предложений, то IDF будет равной: log(10 000 000/1000) = 4. Для расчета окончательного значения веса слова необходимо TF умножить на IDF. В данном примере, TF-IDF вес для слова «заяц» в выбранном документе будет равен: 0,12 × 4 = 0,48.  А дальше мы считаем score для каждого предложения и выбираем предложения с высоким score.  
8. ***Maximum marginal relevance 1998, Carbonell and Goldstein***  
MMR = $argmax_{s_i \in \mathcal{R-S}} (\lambda * Sim_1(s_i, Q) - (1-\lambda) * \max\limits_{s_j \in S}Sim_2(s_i, s_j))$  
S - уже извлеченные предложения из текста  
Q - user query (описание категории документа) ?не очень понятно, где его брать в случае single-document classification   
Sim - similarity metrics  
${\lambda}$ - некий порог, который мы сами устанавливаем  
Идея заключается в том, что мы выбираем предложение, которое не похоже на все остальные в нашем итоговом саммари.   
9. ***MEAD (centric based method) Radev 2000***.  
Для начала определяем наши предложения, как точки в векторном пространстве. Первое, что мы хотим это применить к ним кластеризацию, то есть разделить их на топики. Затем мы ищем центроиды кластеров. И выбираем предложения, которые наиболее близки к центроидам, то есть фактически применяем k-means. Но тут также может возникнуть, что у нас появятся предложения, одинаковые по смыслу, но близкие к центроиду, поэтому нужно будет затем посмотреть на similarity matrix (их виды указаны ниже) и выбрать предложения, у которых коэффицент схожести низкий.  
Имеем предложения в векторном пространстве,делаем клестеризацию по топикам. 
10.  ***LSA (Latent Semantic Analysis)*** [10]  
В основе лежит построение матрицы (слова x предложения), а затем применение сингулярного разложения для этой матрицы. Так, для каждого слова можно получить необходимый вес, а в суммаризацию включать те предложения, чья суммарная "важность" слов наибольшая. 
Итак, опишем подробнее алгоритм.  
Составляем матрицу размерностью(n×m) - term-sentence matrix. В ней каждый ряд соответсвует слову из текстового документа (n слов), а каждая колонка соответсвует предложению из текстового документа (m предложений). Каждый элемент такой матрицы a_ij это вес слова i в предложение j (веса считются по метрике TDIDF, описанной выше). Если в предложении нет слова, то вес этого слова для этого предложения равен 0.  
С помощью алгоритма SVD мы можем определить основные темы нашего документа.  
Применяем SVD для полученной матрицы и получаем три матрицы (U, S, V).  
$A = {U * S * V^T}$  
Матрица U (n×m) представляет ортонормированную матрицу с весами слов  
Матрица S представляет диагональную матрицу (m×m), где каждый ряд i соответсвует весу темы i  
Матрица ${V^T}$ размерностью (n×n) представляет матрицу, где ряд это тема, а колонка это предложение, причем темы расположены в порядке важности. 
Произведение S*V описывает то, насколько предложение соответсвует теме  
Имея такое разложение, как нам понять, какие предложения извлекать?  
     - Gong and Liu (2001). Они использовали матрицу ${V^T}$ для извлечения предложений. Количество предложений, которую нужно выбрать, дано в виде параметра n. Каждый элемент этой матрицы показывает корреляцию между темой и предложением. Итак, в каждой строке мы выбираем наибольший показатель, а потом извлекам в наше краткое описание (реферат) n предложений с высокими показателями.  У данного алгоритма есть недостаток: мы не можем для одного топика выбрать несколько предложений, поэтому спустя несколько лет придумали модификацию к данному способу.  
    - Рзаберемся еще раз подробнее. 𝐴 — матрица терм-предложение, полученная по исходному документу. Её размер равен 𝑛 × 𝑚, где 𝑛 — количество термов в документе, 𝑚 — количество предложений. Элемент 𝑎𝑖𝑗 этой матрицы равен частоте встречаемости терма 𝑖 в тексте, если этот терм встречается в предложении 𝑗, и 0 в противном случае. К полученной матрице применяется сингулярное разложение. 
$A = {U * S * V^T}$  
𝑈 = — ортонормированная матрица размера 𝑛 × 𝑚, 
S = 𝑑𝑖𝑎𝑔(𝜎1, 𝜎2, . . . , 𝜎𝑚) — диагональная матрица,  
𝑉 =  — ортонормированная матрица размера 𝑚 × 𝑚.
Если 𝑟𝑎𝑛𝑘(𝐴) = 𝑟, то выполняется неравенство:  
${\sigma _1 >= \sigma _2>= \sigma _3...>= \sigma _r>= \sigma _r+1 = \sigma _m = ... = 0}$  
С точки зрения семантики сингулярное разложение матрицы 𝐴 интерпретируется как разбиение исходного документа на 𝑟 концепций (тем).
Каждый элемент 𝑣𝑖𝑗 матрицы 𝑉 отражает степень информативности предложения 𝑗 по теме 𝑖. При этом значение 𝜎𝑖 матрицы Σ отражает
степень важности темы 𝑖 в исходном документе. Каждому предложению ${s_k}$ исходного документа присваивается вес по формуле:  
${s_k = \sqrt{\sum\limits_{i=1}^m (v_{ik}) ^2 * \sigma_i^2}}$  
Т.о. больший вес получают предложения, наиболее информативные по одной из тем документа, при этом учитывается и степень важности концепции в документе. Значения весов предложений упорядочиваются по убыванию, и в реферат включаются предложения, соответствующие первым 𝑙 значениям,где 𝑙 — желаемое количество предложений в реферате.  
11. ***LexRank 2004***  
Для начала мы строим similarity matrix, затем представляем каждое предложение в виде вершины графа и смотрим, какие из предложений связаны и насколько их связь сильная по матрице схожести. На графе, если у одной вершины присуствует много ребер, значит6 она схожа со многими другими предложениями, следовательно, ее нужно взять, но отбросить предложения, с которыми она связана. Далее алгоритм PageRank.  
12. ***TextRank***  
TextRank это аналогия PageRank, но для предложений.  
Почти такой же алгоритм, как и LexRank, был изобретен в то же время, но есть маленькие отличия, например, LexRank используется больше для multi-document classification, а TextRank наоборот, для single-document classification + различаются в similarity matrix.  
Он заключается в следующем:  
По тексту строится взвешенный неориентированный граф, вершины в котором обозначают предложения текста. Весом ребра между двумя
вершинами является степень схожести двух предложений, соответствующих вершинам. Она вычисляется, как количество совпадающих слов
в предложениях, нормированное суммарной длиной этих предложений.  
Исходя из весов ребер, каждой вершине присваивается вес по следующей формуле:  
$𝑊({𝑉_i}) = (1 − 𝑑) + 𝑑 * {\sum\limits_{V_j \in Inc(V_i)}\frac{w_{ij}}{\sum\limits_{V_k \in Inc(V_i)}w_{jk}} * W({V_j})}$  
где 𝑉𝑖, 𝑉𝑗 — вершины графа  
𝐼𝑛𝑐(𝑉𝑖) — множество вершин, смежных с вершиной 𝑉𝑖  
𝑤𝑖𝑗 — вес ребра между вершинами 𝑉𝑖, 𝑉𝑗  
𝑑 — коэфициент затухания, который в данном алгоритме равен 0.85  
Итерационный процесс завершается, как только веса вершин перестают меняться более, чем на 0.0001.
После вычисления весов вершин они упорядочиваются по убыванию значения веса и в реферат включаются предложения, соответствующие первым 𝑛 вершинам, где 𝑛 — желаемое количество предложений в кратком описании.  
13. ***ClusterRank***  
 Алгоритм:  
     - Пусть изначально каждое предложение это один отдельный кластер. 
     - Объединим некоторые кластеры, если они схожи друг с другом, схожесть определяем по косинусовой сходимости.  
     - Определяем вес каждого слова в кластере по заданой формуле:  
     W = freq*IDF, где freq - количество предложений в кластере X, в которых присуствует слово w, IDF = $\log \frac{number of sentences}{number of sentences with the word w}$   
      Как мы поймем, что нужно остановиться объединять классы? Изначально мы задаем similarity threshold, на основе которого мы и создаем из двух классов один класс.   
      Основаня идея: выделяются кластеры предложений, описывающих одну идею. Далее из каждого кластера выбирается центроид (самое близкое к центру кластера предложение), которое попадает в финальную суммаризацию. Объединение этих идей - сначала кластеризация предложений и построение графа на основе полученной кластеризации и запуск PageRank для расчета весов.   
https://github.com/AdiChat/senpai  
14. ***Sequencetosequence***   
Это рекуррентная нейронная сеть.
Под капотом у модели находятся энкодер и декодер.  
Энкодер обрабатывает каждый элемент входной последовательности, переводит полученную информацию в вектор, называемый контекстом (context). После обработки всей входной последовательности энкодер пересылает контекст декодеру, который затем начинает генерировать выходную последовательность элемент за элементом.  Но этот подход достаточно медленный, к каждому слову нам нужно применить операцию вычисления. Еще один недостаток заключается в том, что в конец вычисления скрытого состояния должна поместиться вся информация. Поэтому и придумали attention механзм: чем он отличается?  
Во-первых, энкодер передает значительно больше данных декодеру: вместо передачи лишь последнего скрытого состояния после этапа кодирования, энкодер отправляет ему все свои скрытые состояния. Во-вторых, декодер проходит через дополнительный этап, прежде чем сгенерировать выход. Для того, чтобы сфокусироваться на тех частях входной последовательности, которые релевантны для соответствующего временного отрезка.   
В 2018 году появился алгоритм BERT, который показал лучшие результаты к языковым моделям, чем реккурентные нейронные сети и seq2seq модели. 

## Описание тестового набора данных  

Одна из главных и первых задач - это разобраться с данными, на которых мы будем обучать наши модели.  
Какие возможные источники данных мы можем взять? 
1. Субтитры к фильмам и краткие содержания к ним  
Думая о предстоящей второй части работы, алгоритмах распознования речи, датасет с субтитрами и аудиозаписями был бы одним из наиболее подходящих.  
https://github.com/Desklop/Russian_subtitles_dataset
https://yts-subs.com/subtitles/forrest-gump-1994-english-yify-125025
2. Датасет с совещаниями и их кратким саммари.   
Описание датасета:  
Этот датасет включает в себя транскрипты совещаний, их аудиозаписи и краткое содержание совещания. Но главным преимуществом данного датасета является то, что в нем есть разделение на extractive и abstractive. Пример текстового файла: ES2002a.transcript.txt, первые две буквы это страна, в котором записывалось данное совещание, одно совещание состоит из 4 частей: a, b, c и d.  
Данные можно взять отсюда:  
https://github.com/gcunhase/AMICorpusXML/tree/master/data/ami-summary/extractive  

На этапе пробы возьмем один файл из датасета с совещаниями. 

In [ ]:
test = open('/Users/alina/Downloads/AMICorpusXML-master/data/ami-transcripts/ES2002a.transcript.txt', "r", errors = 'ignore')
transcript = test.read()

In [ ]:
transcript

"Um I'm Craig and I'm User Interface. Yeah. Well, my favourite animal would be a monkey. Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them. Yeah. I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house. So um for them it was just how many devices control. Uh.\nMm-hmm. Great. And I'm Andrew and I'm uh our marketing expert. Mm-hmm. Mm-hmm. Yeah, that's that's it. Yeah. I will go. That's fine. Alright. So This one here, right? Okay. Very nice. Alright. My favourite animal is like A beagle. Um charac favourite characteristics of it? Is that right? Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family. And, yeah that they have lots of personality and uh be fit and in robust good health. So this is blue. Blue beagle. My family's beagle. I 

In [ ]:
test2 = open('/Users/alina/Downloads/AMICorpusXML-master/data/ami-summary/extractive/ES2002a.extsumm.txt', "r", errors = 'ignore')
transcript_sum = test2.read()

In [ ]:
transcript_sum

"Um well this is the kick-off meeting for our our project . so we're designing a new remote control and um Um , as you can see it's supposed to be original , trendy and user friendly . Um and so there are three different stages to the design . So we're gonna have like individual work and then a meeting about it . And repeat that process three times . So uh you get to draw your favourite animal and sum up your favourite characteristics of it . My favourite animal is like A beagle . Uh , right , well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family . And , yeah that they have lots of personality Well Then they're small cute and furry , and I kind of like whales . They come in and go eat everything in sight . M my favourite animal is my own dog at home . Um he's very friendly and cheery and always pleased to see you , Um so according to the brief um we're gonna be selling this remote control for twenty five Euro

In [ ]:
print('The length of the transription: {}'.format(len(transcript)))
print('The length of the summary of the transcription: {}'.format(len(transcript_sum)))

The length of the transription: 13676
The length of the summary of the transcription: 4318


In [ ]:
#import os
#folder = '/Users/alina/Downloads/AMICorpusXML-master/data/ami-summary/extractive'
#[x[2] for x in os.walk(folder)]

## Preprocessing

Что включает в себя обработка тектстовых данных (preprocessing)?  
1. Убираем знаки препинания 
2. Лемматизация/токенизация 
3. Убираем лишние символы (числа, знак новой строки, знак почты) 
4. Делим текст на предложения 
5. Делим предложение на отдельные слова
6. Убираем стоп-слова

In [ ]:
# libraries
import re, numpy as np, pandas as pd
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
stopwords_english = stopwords.words('english')
stopwords_english.append('um')
stopwords_english.append('Um')
stopwords_english.append('okay')
stopwords_english.append('uh')
stopwords_english.append('Yeah')
stopwords_english.append('well')
stopwords_english.append('Well')
stopwords_english.append('hmm')
stopwords_english.append('weve')
stopwords_english.append('ive')
stopwords_english.append('mm')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package stopwords to /Users/alina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
a2 = transcript.replace("\n", "")
a = a2.replace("\ ", " ")
a

"Um I'm Craig and I'm User Interface. Yeah. Well, my favourite animal would be a monkey. Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them. Yeah. I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house. So um for them it was just how many devices control. Uh.Mm-hmm. Great. And I'm Andrew and I'm uh our marketing expert. Mm-hmm. Mm-hmm. Yeah, that's that's it. Yeah. I will go. That's fine. Alright. So This one here, right? Okay. Very nice. Alright. My favourite animal is like A beagle. Um charac favourite characteristics of it? Is that right? Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family. And, yeah that they have lots of personality and uh be fit and in robust good health. So this is blue. Blue beagle. My family's beagle. I co

In [ ]:
# убираем знаки препинания из текста 
text =[]
split_regex = re.compile(r'[.|!|?|…]')
sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
for s in sentences:
    text.append(s)

In [ ]:
text

["Um I'm Craig and I'm User Interface",
 'Yeah',
 'Well, my favourite animal would be a monkey',
 "Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them",
 'Yeah',
 'I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house',
 'So um for them it was just how many devices control',
 'Uh',
 'Mm-hmm',
 'Great',
 "And I'm Andrew and I'm uh our marketing expert",
 'Mm-hmm',
 'Mm-hmm',
 "Yeah, that's that's it",
 'Yeah',
 'I will go',
 "That's fine",
 'Alright',
 'So This one here, right',
 'Okay',
 'Very nice',
 'Alright',
 'My favourite animal is like A beagle',
 'Um charac favourite characteristics of it',
 'Is that right',
 'Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family',
 'And, yeah that they have lots of personality and uh be fit a

In [ ]:
# убираем ненужные символы и делим предложение на слова
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent) 

In [ ]:
res = list(sent_to_words(text))

In [ ]:
res

[['um', 'im', 'craig', 'and', 'im', 'user', 'interface'],
 ['yeah'],
 ['well', 'my', 'favourite', 'animal', 'would', 'be', 'monkey'],
 ['then',
  'theyre',
  'small',
  'cute',
  'and',
  'furry',
  'and',
  'uh',
  'when',
  'planet',
  'of',
  'the',
  'apes',
  'becomes',
  'real',
  'im',
  'gonna',
  'be',
  'up',
  'there',
  'with',
  'them'],
 ['yeah'],
 ['know',
  'um',
  'my',
  'parents',
  'went',
  'out',
  'and',
  'bought',
  'um',
  'remote',
  'controls',
  'because',
  'um',
  'they',
  'got',
  'fed',
  'up',
  'of',
  'having',
  'four',
  'or',
  'five',
  'different',
  'remote',
  'controls',
  'for',
  'each',
  'things',
  'the',
  'house'],
 ['so',
  'um',
  'for',
  'them',
  'it',
  'was',
  'just',
  'how',
  'many',
  'devices',
  'control'],
 ['uh'],
 ['mm', 'hmm'],
 ['great'],
 ['and', 'im', 'andrew', 'and', 'im', 'uh', 'our', 'marketing', 'expert'],
 ['mm', 'hmm'],
 ['mm', 'hmm'],
 ['yeah', 'thats', 'thats', 'it'],
 ['yeah'],
 ['will', 'go'],
 ['thats',

In [ ]:
# убираем стоп-слова
res_ = []
s_ =[]
for s in res:
    for word in s:
        if word not in stopwords_english:
            s_.append(word)
    res_.append(s_)
    s_ = []

In [ ]:
res_

[['im', 'craig', 'im', 'user', 'interface'],
 ['yeah'],
 ['favourite', 'animal', 'would', 'monkey'],
 ['theyre',
  'small',
  'cute',
  'furry',
  'planet',
  'apes',
  'becomes',
  'real',
  'im',
  'gonna'],
 ['yeah'],
 ['know',
  'parents',
  'went',
  'bought',
  'remote',
  'controls',
  'got',
  'fed',
  'four',
  'five',
  'different',
  'remote',
  'controls',
  'things',
  'house'],
 ['many', 'devices', 'control'],
 [],
 [],
 ['great'],
 ['im', 'andrew', 'im', 'marketing', 'expert'],
 [],
 [],
 ['yeah', 'thats', 'thats'],
 ['yeah'],
 ['go'],
 ['thats', 'fine'],
 ['alright'],
 ['one', 'right'],
 [],
 ['nice'],
 ['alright'],
 ['favourite', 'animal', 'like', 'beagle'],
 ['charac', 'favourite', 'characteristics'],
 ['right'],
 ['right',
  'basically',
  'high',
  'priority',
  'animal',
  'willing',
  'take',
  'lot',
  'physical',
  'affection',
  'family'],
 ['yeah', 'lots', 'personality', 'fit', 'robust', 'good', 'health'],
 ['blue'],
 ['blue', 'beagle'],
 ['familys', 'beagle']

In [ ]:
# лемматизация 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
res_fin = []
s_fin = []
for s in res_:
    for word in s:
        s_fin.append(lemmatizer.lemmatize(word))
    res_fin.append(s_fin)
    s_fin = []

[nltk_data] Downloading package wordnet to /Users/alina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
res_fin

[['im', 'craig', 'im', 'user', 'interface'],
 ['yeah'],
 ['favourite', 'animal', 'would', 'monkey'],
 ['theyre',
  'small',
  'cute',
  'furry',
  'planet',
  'ape',
  'becomes',
  'real',
  'im',
  'gonna'],
 ['yeah'],
 ['know',
  'parent',
  'went',
  'bought',
  'remote',
  'control',
  'got',
  'fed',
  'four',
  'five',
  'different',
  'remote',
  'control',
  'thing',
  'house'],
 ['many', 'device', 'control'],
 [],
 [],
 ['great'],
 ['im', 'andrew', 'im', 'marketing', 'expert'],
 [],
 [],
 ['yeah', 'thats', 'thats'],
 ['yeah'],
 ['go'],
 ['thats', 'fine'],
 ['alright'],
 ['one', 'right'],
 [],
 ['nice'],
 ['alright'],
 ['favourite', 'animal', 'like', 'beagle'],
 ['charac', 'favourite', 'characteristic'],
 ['right'],
 ['right',
  'basically',
  'high',
  'priority',
  'animal',
  'willing',
  'take',
  'lot',
  'physical',
  'affection',
  'family'],
 ['yeah', 'lot', 'personality', 'fit', 'robust', 'good', 'health'],
 ['blue'],
 ['blue', 'beagle'],
 ['family', 'beagle'],
 ['coul

In [ ]:
# функция, которая объединяет все выше сделанные действия  
def preprocess(filename):
    f = open(filename, "r") 
    a = f.read()
    text =[]
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
    for s in sentences:
        text.append(s)
    res = list(sent_to_words(text))
    res_ = []
    s_ =[]
    for s in res:
        for word in s:
            if word not in stopwords_english:
                s_.append(word)
        res_.append(s_)
        s_ = []
    res_fin = []
    s_fin = []
    for s in res_:
        for word in s:
            s_fin.append(lemmatizer.lemmatize(word))
        res_fin.append(s_fin)
        s_fin = []
    return res_fin

## Similarity metrics (если понадобится) 

1. ***Cosine similarity***
2. ***Jaccard similarity***
3. ***Spearman’s correlation***
4. ***Kendall’s Tau***
5. ***Pearson’s correlation***


## Embeddings (если понадобиться эксперементировать с вариациями embedding)

1. Word2vec. 
2. GloVe (немного быстрее, чем word2vec)
3. Co-occurance matrix 
4. Skip Thought Vector  
5. ELMO (учитывает контекст слова) 
6. FastText

## Метрики 

https://dspace.spbu.ru/bitstream/11701/11076/1/Diploma_Novosyolova_Anastasiia.pdf
https://core.ac.uk/download/pdf/287382511.pdf

Качество оценивается с помощью двух основных метрик BLEU и ROUGE. Можем провести параллель с известными нами метриками (precision и recall).
1. BLEU (машинный перевод -> человеческий текст)  
Основная задача при разработке BLEU состояла в том, чтобы сравнивать N-граммы машинного перевода с N-граммами эталонного перевода и посчитать количество совпадений. Такие совпадения не зависят от порядка слов. Чем их больше, тем лучше предъявляемый перевод.  
Как она считается? Считают максимальное вхождение n-gram из маинного перевода  в человеческий текст и делят на общее количество n-gram в машинном переводе, а затем измеряется средняя точность для N-грамм разной длины (униграммы, биграммы, триграммы и т.д.)

2. ROUGE-метрики (человеческий текст -> машинный перевод, больше для extractive)
ROUGE-N:  
Ситуация, аналогичная ситуации в BLEU, только мы считаем количество совпадаений n-gram из человеческого текста в машинном переводе. 
ROUGE-L:  длина наибольшей общей подпоследовательности между машинным переводом и человеческим текстом  

3. Можно использовать меру F1, чтобы эти показатели работали вместе:
F1 = 2 * (Bleu * Rouge)/(Bleu + Rouge)
4. WER (world error rate). 
$WER = \frac{S + D + I}{N}$, где  
S - количество замен  
D - количество удалений  
I - количество вставок  
N - количество слов  
Это просто расстояние по Левенштейну от того, что мы распознали, до того, что реально было сказано в тексте, поделить на количество слов, реально сказанных в тексте.  
5. METEOR – метрика для оценки перевода с явным контролем, является метрикой для оценки вывода машинного перевода. Эта метрика - модификация метрики BLEU и ROUGE. Опишем подробнее ее алгоритм: 
-Точное установление соответствия — определяются строки, которые являются идентичными в эталонном и машинном переводе. (новое)
-Установление соответствия основ — проводится стемминг (выделение основы слова), и определяются слова с одинаковым корнем в эталонном и машинном переводе.  
-Установление соответствия синонимов — определяются слова, которые являются синонимами в соответствии с WordNet. (новое)  
Дальше вычислим метрику BLEU (P) и метрику ROUGE (R) . Они комбинируются, используя формулу гармонического среднего. 
$F_{mean} = \frac{10PR}{R + 9P}$, а затем определим формулу, которая вычисляется по следующей формуле: 
$p = 0.5 * (\frac{c}{u_{m}}) ^ 3$, где c  - число груп n-gram , а u - количество n-gram, тогда финальный показатель качества: 
M = f(1 - p)

6.  Расстояние Дженсона-Шеннона показывает, сколько информации будет потеряно, если заменить оригинал рефератом. Соответственно, чем ближе показатель к нулю, тем лучше качество.

7. NIST - это метод оценки качества текста, который был переведен с использованием машинного перевода. Он основан на метрике BLEU, но с некоторыми изменениями. Где BLEU просто вычисляет точность nграмм, добавляя равный вес каждому из них, NIST также вычисляет,
насколько информативным является конкретный n-грамм
 То есть, когда найден правильный n-грамм , чем реже, чем n-грамм, тем больше веса
ему будет дано. Например, если биграмм «на» правильно подобран, он будет получать более низкий вес, чем правильное совпадение «интересных вычислений» bigram, поскольку это менее вероятно. NIST также отличается от BLEU в расчете штрафа за краткость, поскольку небольшие вариации в длине перевода не влияют на общий балл  Для нашей задачи подходит метрика BLEU, изначально она создавалась на основе показателя вероятности ошибок, используемого для оценки качества распознавания речи. 

In [ ]:
# загружаем нужные нам библиотеки 
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
punctuation += '\n'
from heapq import nlargest
from nltk.cluster.util import cosine_distance
import networkx as nx
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from collections import Counter

## Extractive methods 

### Word frequency method  

Попробуем применить самый простой подход, основанный на частоте слов, пошагово. Что мы будем делать? Мы будем смотреть на то, как часто слово встречается в параграфе, например, если слово "cool" появилось в предложении 5 раз, то его вес будет равняться 5.  Так мы сможем создать score для каждого предложения, но в score каждого предложения мы будем брать нормализированный вес слова, то есть его частоту, деленную на длину предложения. Тем самым, в конце мы выберем 

In [ ]:
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')
doc = nlp(a)
tokens = [token.text for token in doc]
tokens

['Um',
 'I',
 "'m",
 'Craig',
 'and',
 'I',
 "'m",
 'User',
 'Interface',
 '.',
 'Yeah',
 '.',
 'Well',
 ',',
 'my',
 'favourite',
 'animal',
 'would',
 'be',
 'a',
 'monkey',
 '.',
 'Then',
 'they',
 "'re",
 'small',
 'cute',
 'and',
 'furry',
 ',',
 'and',
 'uh',
 'when',
 'planet',
 'of',
 'the',
 'apes',
 'becomes',
 'real',
 ',',
 'I',
 "'m",
 'gon',
 'na',
 'be',
 'up',
 'there',
 'with',
 'them',
 '.',
 'Yeah',
 '.',
 'I',
 'know',
 'um',
 'My',
 'parents',
 'went',
 'out',
 'and',
 'bought',
 'um',
 'remote',
 'controls',
 'because',
 'um',
 'they',
 'got',
 'fed',
 'up',
 'of',
 'having',
 'four',
 'or',
 'five',
 'different',
 'remote',
 'controls',
 'for',
 'each',
 'things',
 'the',
 'house',
 '.',
 'So',
 'um',
 'for',
 'them',
 'it',
 'was',
 'just',
 'how',
 'many',
 'devices',
 'control',
 '.',
 'Uh',
 '.',
 'Mm',
 '-',
 'hmm',
 '.',
 'Great',
 '.',
 'And',
 'I',
 "'m",
 'Andrew',
 'and',
 'I',
 "'m",
 'uh',
 'our',
 'marketing',
 'expert',
 '.',
 'Mm',
 '-',
 'hmm',
 '

In [ ]:
word_freq = {}
for word in doc:
    if word.text.lower() not in stopwords_english:
        if word.text.lower() not in punctuation:
            if word.text not in word_freq.keys():
                word_freq[word.text] = 1
            else:
                word_freq[word.text] += 1

In [ ]:
print(word_freq)

{"'m": 17, 'Craig': 2, 'User': 1, 'Interface': 1, 'Yeah': 39, 'favourite': 6, 'animal': 7, 'would': 11, 'monkey': 1, "'re": 18, 'small': 2, 'cute': 1, 'furry': 1, 'planet': 1, 'apes': 1, 'becomes': 1, 'real': 1, 'gon': 12, 'na': 12, 'know': 26, 'parents': 1, 'went': 1, 'bought': 2, 'remote': 22, 'controls': 6, 'got': 7, 'fed': 2, 'four': 1, 'five': 6, 'different': 6, 'things': 4, 'house': 2, 'many': 2, 'devices': 1, 'control': 17, 'Great': 2, 'Andrew': 2, 'marketing': 3, 'expert': 1, "'s": 51, 'go': 5, 'fine': 1, 'Alright': 6, 'one': 10, 'right': 9, 'nice': 1, 'like': 41, 'beagle': 3, 'charac': 1, 'characteristics': 2, 'basically': 1, 'high': 2, 'priority': 1, 'willing': 1, 'take': 3, 'lot': 4, 'physical': 1, 'affection': 1, 'family': 3, 'yeah': 19, 'lots': 2, 'personality': 1, 'fit': 1, 'robust': 1, 'good': 7, 'health': 1, 'blue': 1, 'Blue': 1, 'coulda': 1, 'told': 1, 'whole': 1, 'beagles': 1, 'Boy': 1, 'let': 1, 'tell': 1, 'Impressionist': 1, 'Superb': 1, 'sketch': 1, 'way': 3, 'Yep'

In [ ]:
max_freq = max(word_freq.values())
for word in word_freq.keys():
    word_freq[word] = word_freq[word]/max_freq

In [ ]:
sentence_tokens = [sent for sent in doc.sents]

In [ ]:
sentence_tokens

[Um I'm Craig,
 and I'm User Interface.,
 Yeah.,
 Well, my favourite animal would be a monkey.,
 Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them.,
 Yeah.,
 I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house.,
 So um for them it was just how many devices control.,
 Uh.,
 Mm-hmm.,
 Great.,
 And I'm Andrew,
 and I'm uh our marketing expert.,
 Mm-hmm.,
 Mm-hmm.,
 Yeah, that's that's it.,
 Yeah.,
 I will go.,
 That's fine.,
 Alright.,
 So This one here, right?,
 Okay.,
 Very nice.,
 Alright.,
 My favourite animal is like A beagle.,
 Um charac favourite characteristics of it?,
 Is that right?,
 Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family.,
 And, yeah that they have lots of personality and uh be fit and in robust good healt

In [ ]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_freq.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_freq[word.text.lower()]
            else:
                sentence_scores[sent] += word_freq[word.text.lower()]

In [ ]:
print(sentence_scores)

{Um I'm Craig: 0.3333333333333333, and I'm User Interface.: 0.39215686274509803, Yeah.: 0.37254901960784315, Well, my favourite animal would be a monkey.: 0.49019607843137253, Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them.: 1.3137254901960784, Yeah.: 0.37254901960784315, I know um My parents went out and bought um remote controls because um they got fed up of having four or five different remote controls for each things the house.: 2.235294117647059, So um for them it was just how many devices control.: 0.39215686274509803, And I'm Andrew: 0.3333333333333333, and I'm uh our marketing expert.: 0.4117647058823529, Yeah, that's that's it.: 2.372549019607843, Yeah.: 0.37254901960784315, I will go.: 0.09803921568627451, That's fine.: 1.0196078431372548, So This one here, right?: 0.37254901960784315, Very nice.: 0.0196078431372549, My favourite animal is like A beagle.: 1.1176470588235294, Um charac favourite characteristics o

In [ ]:
select_length = int(len(sentence_tokens) * 0.05)

In [ ]:
summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
summary

[that doesn't make it any better, but that just the appeal of of not having You know, these days there's a r pe things in people's homes are becoming more and more like chic, you know.,
 Well twenty five Euro, I mean that's um that's about like eighteen pounds or something, isn't it?,
 Um so anything else anybody wants to add about what they don't like about remote controls they've used, what they would really like to be part of this new one at all?,
 Or just like, basic product podi positioning, the twenty five Euro remote control might be a big hit in London, might not be such a big hit in Greece, who knows, something like that,,
 And um, you know, when I think about what they are now, it's better, but actually it's still kind of,,
 Um, I dunno, I've never bought a remote control, so I don't know how how good a remote control that would get you.,
 Actually had a cable between it and the T_V_ and big like buttons that sort of like, like on a blender or something.,
 Yeah, interesting t

In [ ]:
summary_luhn =[]
for i in summary:
    summary_luhn.append(str(i))
final_luhn = ' '.join(summary_luhn)
final_luhn 

"that doesn't make it any better, but that just the appeal of of not having You know, these days there's a r pe things in people's homes are becoming more and more like chic, you know. Well twenty five Euro, I mean that's um that's about like eighteen pounds or something, isn't it? Um so anything else anybody wants to add about what they don't like about remote controls they've used, what they would really like to be part of this new one at all? Or just like, basic product podi positioning, the twenty five Euro remote control might be a big hit in London, might not be such a big hit in Greece, who knows, something like that, And um, you know, when I think about what they are now, it's better, but actually it's still kind of, Um, I dunno, I've never bought a remote control, so I don't know how how good a remote control that would get you. Actually had a cable between it and the T_V_ and big like buttons that sort of like, like on a blender or something. Yeah, interesting thing about dis

In [ ]:
rouge = Rouge()
scores = rouge.get_scores(final_luhn, transcript_sum)
print('Luhn scores: {}'.format(scores))

Luhn scores: [{'rouge-1': {'f': 0.5059101609789358, 'p': 0.7379310344827587, 'r': 0.38489208633093525}, 'rouge-2': {'f': 0.28097868531433284, 'p': 0.41013824884792627, 'r': 0.21368547418967587}, 'rouge-l': {'f': 0.467289714841401, 'p': 0.589622641509434, 'r': 0.38699690402476783}}]


In [ ]:
'''
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)
'''

1.0


**Возможные улучшения**: 
1. Заметим, что саммари, которое мы получили, не связано и не описывает основную суть фильма. В нем содержится много лишних слов, таких как stuff, goddamn и другое. Эти слова не несут в себе смысла, отражающего содеражние фильма, вероятно, чтобы улучшить данный метод,надо удалять слова, которые встречаются в нашем тексте чаще всего, чтобы их вес не мешал нам составлять саммари. 
2. Так как у нас ведется диалог между людьми, то в саммари должна происходить некая замена прямой речи на косвенную речь, но тогда нужно подумать, как быть с тем, что у нас нет информации о том, кто что говорит. 
3. Чтобы избежать "я" нужно не брать их во внимания, наша модель должна запоминать основные факты слова и в идеале дополнять предложения, но это уже abstractive модели. 

### TextRank

1. Сплитим текст о предложениям  
2. Считаем похожесть предложений между собой  
3. Строим граф предложений с взвешенными ребрами  
4. С помощью PageRank ищем наиболее важные предложенмия 

Как работает алгоритм PageRank в нашем случае?  
У нас есть similarity матрица, к ней нужно применить PageRank. 
Принято считать, что на нулевой (0) итерации PageRank каждой страницы одинаковый и равен 1 / N. Далее мы применяем ниже написанную формулу, получаем следующий вектор, который показывает веса каждой вершины нашего графа, затем повторяем эту операцию до тех пор, пока мы не достигнем состояния convergence. На следующих итерациях используется вес всех входящих ссылок, который представляет собой вес с предыдущей итерации делённый на количество исходящих ссылок (в формуле – L).
PR (A) = (1 - d) / N + d * (PR(B) / L(B) + PR(C) / L(C) + ...)
N – общее количество вершин;
d – коэффициент затухания (обычно используется значение 0,85);
L – количество исходящих ребер.

http://dnevniknauki.ru/images/publications/2020/5/technics/Gritsenko.pdf  
https://arxiv.org/pdf/1602.03606.pdf  

TextRank это аналогия PageRank, но для предложений.  
Предложение - node графа. 
Почти такой же алгоритм, как и LexRank, был изобретен в то же время, но есть маленькие отличия, например, LexRank используется больше для multi-document classification, а TextRank наоборот, для single-document classification + различаются в similarity matrix.  
Он заключается в следующем:  
По тексту строится взвешенный неориентированный граф, вершины в котором обозначают предложения текста. Весом ребра между двумя
вершинами является степень схожести двух предложений, соответствующих вершинам. Она вычисляется, как количество совпадающих слов
в предложениях, нормированное суммарной длиной этих предложений.  
Исходя из весов ребер, каждой вершине присваивается вес по следующей формуле:  
$𝑊({𝑉_i}) = (1 − 𝑑) + 𝑑 * {\sum\limits_{V_j \in Inc(V_i)}\frac{w_{ij}}{\sum\limits_{V_k \in Inc(V_i)}w_{jk}} * W({V_j})}$  
где 𝑉𝑖, 𝑉𝑗 — вершины графа  
𝐼𝑛𝑐(𝑉𝑖) — множество вершин, смежных с вершиной 𝑉𝑖  
𝑤𝑖𝑗 — вес ребра между вершинами 𝑉𝑖, 𝑉𝑗  
𝑑 — коэфициент затухания, который в данном алгоритме равен 0.85  
Итерационный процесс завершается, как только веса вершин перестают меняться более, чем на 0.0001.
После вычисления весов вершин они упорядочиваются по убыванию значения веса и в реферат включаются предложения, соответствующие первым 𝑛 вершинам, где 𝑛 — желаемое количество предложений в кратком описании.  

In [ ]:
# косинусовая сходимость 
# на вход подается списки слов двух предложений
def read(filename):
    f = open(filename, "r", errors = 'ignore') 
    a = f.read()
    text =[]
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
    for s in sentences:
        text.append(s)
    res = list(sent_to_words(text))
    return res


def sentence_similarity(sent1, sent2, stopwords):
    all_words = list(set(sent1 + sent2))
     
    # вектор для первого предложения 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # вектор для второго предложения 
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return (1 - cosine_distance(vector1, vector2))


# создаем матрицу косинусовой сходимости 
def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix


def generate_summary_textrank(file_name, top_n):
    summarize_text = []
    sentences = read(file_name)
    sentence_similarity_martix = build_similarity_matrix(sentences, stopwords_english)

    # используем pagerank, чтобы определить веса предложений 
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank_numpy(sentence_similarity_graph)

    # отсортируем предложения и выберем веса с большим весом 
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n - 1):
        if len(ranked_sentence[i][1]) > 2 and len(list((Counter(ranked_sentence[i][1]) - Counter(ranked_sentence[i + 1][1])).elements())) > 3:
              summarize_text.append(" ".join(ranked_sentence[i][1]))
    return summarize_text
    

In [ ]:
a = generate_summary_textrank('/Users/alina/Downloads/AMICorpusXML-master/data/ami-transcripts/ES2002a.transcript.txt', top_n=100)
print("Summarize Text TextRank: \n", ". ".join(a))

/Users/alina/opt/anaconda3/lib/python3.7/site-packages/nltk/cluster/util.py:131: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


Summarize Text TextRank: 
 um yeah and kind of like whales. so like wonder if we might add something new to the to the remote control market such as the lighting in your house or um yeah yeah. like how much does you know remote control cost. right so do you think that should be like main design aim of our remote control you know do your your satellite and your regular telly and your v_c_r_ and everything. and um you know when think about what they are now its better but actually its still kind of dunno like massive junky thing on the table. um guess thats up to us mean you probably want some kind of unique selling point of it so um you know yeah. yeah thats thats it. um dunno ive never bought remote control so dont know how how good remote control that would get you. my favourite animal is like beagle. oh yeah regions and stuff yeah. do we have any other background information on like how that compares to other other yeah. um so anything else anybody wants to add about what they dont l

Применив алгоритм TextRank, можно заметить, что наше краткое содержание не отражает главную суть текста и включает в себя предложения из одного-двух слов, которые почти одни и те же (forrest gump). Давайте попробуем поставить ограничение на то, что предложение должно содержать больше 4 - 5 слов и эти слова должны быть не похожи на лругие слова в нашем саммари, поэтому в конце добавим условия на то, что длина предложения > 2 и слова одного предложения отличаются от слов второго предложения на 3 и больше. 
Возможные варианты модификации данного алгоритма: 
1. Мы столкнулись с проблемой, что предложения в саммари одинаковые, поэтому возможно нужно добавить оглраничение на добавление предложения в саммари. Давайте попробуем использовать Maximum marginal relevance из пункта 8 (описание возможных алгоритмов)    
MMR = $argmax_{s_i \in \mathcal{R-S}} (\lambda * Sim_1(s_i, Q) - (1-\lambda) * \max\limits_{s_j \in S}Sim_2(s_i, s_j))$  


In [ ]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
textrank_hypothesis = ''
for i in a:
    textrank_hypothesis += i 

In [ ]:
from rouge import Rouge 
rouge = Rouge()
scores = rouge.get_scores(textrank_hypothesis, transcript_sum)
print('TextRank scores: {}'.format(scores))

TextRank scores: [{'rouge-1': {'f': 0.5923261340887291, 'p': 0.592326139088729, 'r': 0.592326139088729}, 'rouge-2': {'f': 0.3157262855162065, 'p': 0.3157262905162065, 'r': 0.3157262905162065}, 'rouge-l': {'f': 0.48795180223259005, 'p': 0.4750733137829912, 'r': 0.5015479876160991}}]


### ClusterRank

https://github.com/AdiChat/senpai

In [ ]:
def read(filename):
    f = open(filename, "r", errors = 'ignore') 
    a = f.read()
    text =[]
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
    for s in sentences:
        text.append(s)
    res = list(sent_to_words(text))
    return res

# Расстояние левенштайна 
def lDistance(firstString, secondString):
    if len(firstString) > len(secondString):
        firstString, secondString = secondString, firstString
    distances = range(len(firstString) + 1)
    for index2, char2 in enumerate(secondString):
        newDistances = [index2 + 1]
        for index1, char1 in enumerate(firstString):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
               newDistances.append(1 + min((distances[index1], distances[index1+1], newDistances[-1])))
        distances = newDistances
    return distances[-1]

def build_lev_matrix(sentences, stop_words):
    lev_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            lev_matrix[idx1][idx2] = lDistance(sentences[idx1], sentences[idx2])
    return lev_matrix


def generate_clusterrank_summary(file_name, top_n):
    summarize_text = []
    sentences = read(file_name)
    sentence_lev_martix = build_lev_matrix(sentences, stopwords_english)

    # используем pagerank, чтобы определить веса предложений 
    sentence_lev_graph = nx.from_numpy_array(sentence_lev_martix)
    scores = nx.pagerank(sentence_lev_graph)

    # отсортируем предложения и выберем веса с большим весом 
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    return summarize_text

In [ ]:
k = generate_clusterrank_summary('/Users/alina/Downloads/AMICorpusXML-master/data/ami-transcripts/ES2002a.transcript.txt', 10)
print("Summarize Text: \n", ". ".join(k))

Summarize Text: 
 im thinking the price might might appeal to certain market in one region whereas in another itll be different so just chara just characteristic of the just or just like basic product podi positioning the twenty five euro remote control might be big hit in london might not be such big hit in greece who knows something like that yeah. cause it could it could it could be that it could be that functionally that doesnt make it any better but that just the appeal of of not having you know these days theres pe things in peoples homes are becoming more and more like chic you know. um hes very friendly and cheery and always pleased to see you and very kind of affectionate and um uh and hes quite quite wee as well so you know he can doesnt take up too much space. yeah interesting thing about discussing um production of remote control for me is that as you point out just dont think of remote controls as somethin something people consciously assess in their purchasing habits. um 

In [ ]:
k

['im thinking the price might might appeal to certain market in one region whereas in another itll be different so just chara just characteristic of the just or just like basic product podi positioning the twenty five euro remote control might be big hit in london might not be such big hit in greece who knows something like that yeah',
 'cause it could it could it could be that it could be that functionally that doesnt make it any better but that just the appeal of of not having you know these days theres pe things in peoples homes are becoming more and more like chic you know',
 'um hes very friendly and cheery and always pleased to see you and very kind of affectionate and um uh and hes quite quite wee as well so you know he can doesnt take up too much space',
 'yeah interesting thing about discussing um production of remote control for me is that as you point out just dont think of remote controls as somethin something people consciously assess in their purchasing habits',
 'um and 

In [ ]:
clusterrank_hypothesis = ''
for i in k:
    clusterrank_hypothesis += i

In [ ]:
rouge = Rouge()
scores = rouge.get_scores(clusterrank_hypothesis, transcript_sum)
scores

[{'rouge-1': {'f': 0.4539363441966954,
   'p': 0.7527777777777778,
   'r': 0.3249400479616307},
  'rouge-2': {'f': 0.22651006290472558,
   'p': 0.37604456824512533,
   'r': 0.16206482593037214},
  'rouge-l': {'f': 0.42043221540151543,
   'p': 0.5752688172043011,
   'r': 0.33126934984520123}}]

In [ ]:
1. Нужно убрать из саммари стоп-слова
2. Есть почему-то повторяющиеся слова, которые идут друг за другом 

### LSA

Источники, где можно посмотреть  про этот метод:  
https://github.com/llazzaro/lsa_python/blob/master/lsa/lsa.py  
https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python  
http://www.kiv.zcu.cz/~jstein/publikace/isim2004.pdf  
https://rare-technologies.com/text-summarization-in-python-extractive-vs-abstractive-techniques-revisited/

### Вручную создаем матрицу частот td-idf

1. Для начала создадим матрицу, где каждый элемент это частота слова в предложении. через td-idf.

In [ ]:
# алгоритм для построения матрицы td-idf вручную 
'''
import collections
DF = {}
for i in range(len(res_fin)):
    tokens = res_fin[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

c = collections.Counter()
tf_idf = {}
N = len(res_fin)
for i in range(N):
    sentence = res_fin[i]
    for i in sentence:
        c[i] += 1
    #counter = Counter(words)
    for word in np.unique(words):
        tf = c[word]/len(sentence)
        #df = doc_freq(token)
        idf = np.log(N/DF[word])
        tf_idf[sentence, word] = tf*idf
'''

'\nimport collections\nDF = {}\nfor i in range(len(res_fin)):\n    tokens = res_fin[i]\n    for w in tokens:\n        try:\n            DF[w].add(i)\n        except:\n            DF[w] = {i}\nfor i in DF:\n    DF[i] = len(DF[i])\n\nc = collections.Counter()\ntf_idf = {}\nN = len(res_fin)\nfor i in range(N):\n    sentence = res_fin[i]\n    for i in sentence:\n        c[i] += 1\n    #counter = Counter(words)\n    for word in np.unique(words):\n        tf = c[word]/len(sentence)\n        #df = doc_freq(token)\n        idf = np.log(N/DF[word])\n        tf_idf[sentence, word] = tf*idf\n'

### Созданием матрицы td-idf через библиотеку

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def calc_sigma(matrix):
    (m, n) = matrix.shape   # узнаем размерность матрицы train
    sigma = np.zeros((m, n))  # задаем матрицу из нулей для сигмы такой же размерности, как A
 
    matrix_transposed = matrix.transpose()    # транспонируем A
    matrix_c = 1/n * matrix.dot(matrix_transposed)   # считаем ковариационную матрицу C
 
    wa, u = np.linalg.eig(matrix_c)   # wa - собственные значения, u - соответствующие собственные векторы
    wa.sort()   # сортируем собственные значения
    wa = wa[::-1]  # в порядке убывания
 
    sigmas = np.sqrt(wa)   # вычисляем корни от всех собственных чисел
 
    i = 0
    if m < n:   # выставляем собственные значения по главной диагонали
        while i < m:
            sigma[i, i] = sigmas[i]
            i += 1
    if n < m:
        while i < n:
            sigma[i, i] = sigmas[i]
            i += 1
 
    s = []  # формальность: для удобства меняем тип array на список
    for vector in sigma:
        s.append([vector[i] for i in range(len(vector))])
 
    return s
 
 
# Подсчет W_transposed - каждый столбик - собственный вектор матрицы C = A^(T) * A (отсортированы по убыванию)
 
def calc_w(matrix):
    va_sorted = []  # отсортируем собственные векторы по убыванию собственных значений
 
    matrix_transposed = matrix.transpose()  # транспонируем A
    matrix_c = matrix_transposed.dot(matrix)  # считаем матрицу С
    print("Матрица C: ")
    print(matrix_c)
    print()
 
    eigenvalues_c, eigenvectors_c = np.linalg.eig(matrix_c)  # собственные значения и собственные векторы матрицы C (правые)
    wa_index_value = []  # создадим массив с элементами вида (индекс собственного значения, собственное значение)
 
    counter = 0
    while counter < len(eigenvalues_c):  # заполним такой массив
        (m, n) = (counter, eigenvalues_c[counter])
        wa_index_value.append((m, n))
        counter += 1
    wa_index_value_sorted = sorted(wa_index_value, key=lambda index_value: index_value[1])  # сортируем элементы по величине собственного значения
    wa_index_value_sorted = wa_index_value_sorted[::-1]  # в порядке убывания
 
    counter = 0  # заполняем поэлементно матрицу сортированных собственных значений
    while counter < len(wa_index_value_sorted):  # проходим по очереди по всем () в wa_index_value_sorted
        va_sorted.append(eigenvectors_c[:, wa_index_value_sorted[counter][0]])  # берем из списка собств. векторов тот, который с таким же индексом, как индекс в ()
        counter += 1
 
    eigenvectors_c_sorted = []  # формальность: для удобства меняем тип array на список
    for vector in va_sorted:
        eigenvectors_c_sorted.append([vector[i] for i in range(len(va_sorted))])
 
    w = np.array(eigenvectors_c_sorted).transpose()  # транспонируем матрицу W
 
    return w
 
 
#   Подсчет U
 
def calc_u(matrix):
    matrix_transposed = matrix.transpose()  # транспонируем A
    matrix_c2 = matrix.dot(matrix_transposed)  # считаем матрицу c2
    va_sorted = []  # отсортируем впоследствии собственные векторы по убыванию собственных значений и внесем сюда
 
    eigenvalues_c2, eigenvectors_c2 = np.linalg.eig(matrix_c2)   # левые собственные числа и векторы
    wa_index_value = []  # создадим массив с элементами вида (индекс собственного значения, собственное значение)
 
    counter = 0
    while counter < len(eigenvalues_c2):  # заполним такой массив
        (m, n) = (counter, eigenvalues_c2[counter])
        wa_index_value.append((m, n))
        counter += 1
    wa_index_value_sorted = sorted(wa_index_value, key=lambda index_value: index_value[1])  # сортируем элементы по величине собственного значения
    wa_index_value_sorted = wa_index_value_sorted[::-1]  # в порядке убывания
 
    counter = 0  # заполняем поэлементно матрицу сортированных собственных значений
    while counter < len(wa_index_value_sorted):  # проходим по очереди по всем () в wa_index_value_sorted
        va_sorted.append(eigenvectors_c2[:, wa_index_value_sorted[counter][0]])  # берем из списка собств. векторов тот, который с таким же индексом, как индекс в ()
        counter += 1
 
    u = []  # формальность: для удобства меняем тип array на список
    for vector in va_sorted:
        u.append([vector[i] for i in range(len(va_sorted))])
    u = np.array(u)
 
    return u

In [ ]:
# входные данные (текст , разбитый на предложения; количество предложений, которое мы хотим видеть в summary)
def summary_LSA(text, k):
    vectorizer = TfidfVectorizer(stop_words = stopwords_english)
    X = vectorizer.fit_transform(text)
    y = X.toarray()
    a = np.matrix(y).T
    U_numpy, s_numpy, W_numpy = np.linalg.svd(a) 
    W_numpy_2 = np.square(W_numpy)
    s_numpy_2 = np.square(s_numpy)
    weights = np.sqrt(np.dot(W_numpy_2, s_numpy_2))
    weights = np.array(weights)[0]
    dict_weights = {k: v for k, v in enumerate(list(weights))}
    indexes_weights = sorted(dict_weights, key=dict_weights.get, reverse = True)
    indexes = indexes_weights[:k]
    summary = []
    for i in indexes:
        summary.append(text[i])
    return summary
    

In [ ]:
summary = summary_LSA(text, 50)

/Users/alina/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['yeah'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
print("Summarize Text LSA: \n", ". ".join(summary))

Summarize Text LSA: 
 So cost like production cost is twelve fifty, but selling price is is that wholesale or retail. Think it will. I imagine it probably is our sale actually because it's probably up to the the um the retailer to uh sell it for whatever price they want. Ah. Yeah, yeah. Okay, uh we now need to discuss the project finance. Um and what do I like about him, um That's just to suggest that his tail wags. Um, can we just go over that again. Is he aware that th it's his own cha tail he's chasing. Uh um. Um so according to the brief um we're gonna be selling this remote control for twenty five Euro, um and we're aiming to make fifty million Euro. I don't know what mine is. All together. And uh we don't want it to cost any more than uh twelve fifty Euros, so fifty percent of the selling price. Yeah. Well like um, maybe what we could use is a sort of like a example of a successful other piece technology is palm palm pilots. Do we have any other background information on like how

In [ ]:
lsa_hypothesis = ''
for i in summary:
    lsa_hypothesis += i
rouge = Rouge()
scores = rouge.get_scores(lsa_hypothesis, transcript_sum)
scores

[{'rouge-1': {'f': 0.4440789430592647,
   'p': 0.7068062827225131,
   'r': 0.3237410071942446},
  'rouge-2': {'f': 0.2487644108496292,
   'p': 0.3963254593175853,
   'r': 0.18127250900360145},
  'rouge-l': {'f': 0.3724394737907334,
   'p': 0.4672897196261682,
   'r': 0.30959752321981426}}]

In [ ]:
'''
import nltk
hypothesis = ['It', 'is', 'a', 'cat', 'inside','room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)
'''

0.6731821382417487


### KLsum (нужно немного улучшить содеражние саммари, поставить ограничение на количество предложений)

Вероятностный алгоритм, который используя KL дивергенцию как меру расстояния между вероятностными распределениями, ищет подмножество предложений текста, распределение слов в которых ближе всего к распределению слов в исходном тексте.KL Sum вводит следующий критерий для выбора предложений в реферат S для предложения D:  
KL(P|Q) = Σ P(w)log P(w)/Q(w)  
Это метод, который добавляет предложения, пока коэффицент Kl divergence уменьшается. Цель этого алгорттма найти множество предложений, чья длина слов меньше L и распределение слов в саммари совпадает с распределением в тексте.  


In [ ]:
# принимает список частот слов в саммари и список частот слов в тексте
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent) 

def kl_divergence(summary_density, text_density):
        sum_val = 0
        # для каждого слова из саммари смотрим, есть ли слово из саммари в тексте 
        for w in summary_density:
            # frequency 
            frequency = text_density.get(w)
            # если частота этого слова существует, то мы считаем частоту в тексте и в саммари этого слова и используем формулу kl
            if frequency: 
                sum_val += frequency * math.log(frequency / summary_density[w])
        return sum_val


def get_all_words(snw):
    all_words = []
    for i in snw:
        for j in i:
            all_words.append(j)
    return all_words

def calculate_freq(array):
    word_freq = {}
    for w in array:
        word_freq[w] = word_freq.get(w, 0) + 1
    return word_freq  

def get_sentences_list(filename):
    f = open(filename, "r", errors = 'ignore') 
    a = f.read()
    text =[]
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
    for s in sentences:
        text.append(s)
    return text   

def find_index_of_best_sentence(kls):
    return kls.index(min(kls))

In [ ]:
'''
import math 
import re
from gensim import models
sentences_list = get_sentences_list('/Users/alina/Downloads/Forrest_Gump-English.txt')
sentences_as_words = list(sent_to_words(sentences_list))
summary = []
all_w = get_all_words(sentences_as_words)
word_freq = calculate_freq(all_w)
while len(sentences_list) > 0:
    # будет хранить значения kls, чтобы из них выбрать наиментгее
    kls = []
    # превращает краткое содержание в список слов 
    summary_as_word_list = get_all_words(summary)

    for s in sentences_as_words:
        # если мы добавим предложение, то какое будет распределение
        # соединим список слов в предложении и то, что в саммари 
        new_joint = s + summary_as_word_list
        joint_freq = calculate_freq(new_joint)

        # считаем kls между распределением саммари и слова в тексте 
        kls.append(kl_divergence(joint_freq, word_freq))

    indexToRemove = find_index_of_best_sentence(kls)
    best_sentence = sentences_list.pop(indexToRemove)
    del sentences_as_words[indexToRemove]
    summary.append(best_sentence)
'''

"\nimport math \nimport re\nfrom gensim import models\nsentences_list = get_sentences_list('/Users/alina/Downloads/Forrest_Gump-English.txt')\nsentences_as_words = list(sent_to_words(sentences_list))\nsummary = []\nall_w = get_all_words(sentences_as_words)\nword_freq = calculate_freq(all_w)\nwhile len(sentences_list) > 0:\n    # будет хранить значения kls, чтобы из них выбрать наиментгее\n    kls = []\n    # превращает краткое содержание в список слов \n    summary_as_word_list = get_all_words(summary)\n\n    for s in sentences_as_words:\n        # если мы добавим предложение, то какое будет распределение\n        # соединим список слов в предложении и то, что в саммари \n        new_joint = s + summary_as_word_list\n        joint_freq = calculate_freq(new_joint)\n\n        # считаем kls между распределением саммари и слова в тексте \n        kls.append(kl_divergence(joint_freq, word_freq))\n\n    indexToRemove = find_index_of_best_sentence(kls)\n    best_sentence = sentences_list.pop(

In [ ]:
# существует библиотека для этого метода 
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.kl import KLSummarizer
document = a
parser=PlaintextParser.from_string(document,Tokenizer("english"))
summarizer = KLSummarizer()
summary = summarizer(parser.document,50)
summ_klsumm = []
for sentence in summary:
    summ_klsumm.append(sentence)

In [ ]:
print('Summarization KLSum: {}'.format(summ_klsumm))

Summarization KLSum: [<Sentence: ['um yeah and kind of like whales', 'so like wonder if we might add something new to the to the remote control market such as the lighting in your house or um yeah yeah', 'like how much does you know remote control cost', 'right so do you think that should be like main design aim of our remote control you know do your your satellite and your regular telly and your v_c_r_ and everything', 'and um you know when think about what they are now its better but actually its still kind of dunno like massive junky thing on the table', 'um guess thats up to us mean you probably want some kind of unique selling point of it so um you know yeah', 'yeah thats thats it', 'um dunno ive never bought remote control so dont know how how good remote control that would get you', 'my favourite animal is like beagle', 'oh yeah regions and stuff yeah', 'do we have any other background information on like how that compares to other other yeah', 'um so anything else anybody wants

### Mead

Для начала определяем наши предложения, как точки в векторном пространстве. Первое, что мы хотим это применить к ним кластеризацию, то есть разделить их на топики. Затем мы ищем центроиды кластеров. И выбираем предложения, которые наиболее близки к центроидам, то есть фактически применяем k-means. Но тут также может возникнуть, что у нас появятся предложения, одинаковые по смыслу, но близкие к центроиду, поэтому нужно будет затем посмотреть на similarity matrix (их виды указаны ниже) и выбрать предложения, у которых коэффицент схожести низкий.  
Имеем предложения в векторном пространстве,делаем клестеризацию по топикам. 

Итак, определим точки в векторном пространстве. Как мы это сделаем? Мы создадим множество слов нашего документа, и они будут нашими столбцами, а наши рядами будут сами предложения, получается, у нас будет матрица из n строк, где n - число предложений и из m столбцов, где m - количество уникальных слов в документе.  
Итак, теперь нам нужно применить KMeans, но центроид в алгоритме Kmeans это центроид масс, то есть это среднее значение векторов, описывающих это класс. А что такое среднее значение слов предложений? С семантической точки зрения брать среднее значение бессмысленно. Поэтому нужно придумать другой метод, по сути скорее всего в нашей матрице данных у нас будут в основном бинарные данные за исключением редких случаев. Может, тогда есть смысл считать расстояние Хемминга и сделать каким-то образом кластеризацию с другой метрикой. Но, к счастью, есть такой способ Kmedoids (https://en.wikipedia.org/wiki/K-medoids), k-means требует в качестве метрики евклидово расстояние, а этот метод минимизирует сумма парных схожестей между 2 объектами.  
Медоид (в кластерном анализе) — объект, принадлежащий набору данных или кластеру, различие (например, по координатам) которого с другими объектами в наборе данных или кластере минимально. Медоиды близки по смыслу центроидам, но в отличие от них, являются объектом, принадлежащим кластеру, и как правило используются в тех случаях, когда невозможно вычислить средние координаты или центр масс кластера. И это то, что нам нужно!  
Количество кластеров - колчиество предложений, которые мы хотим изъять в краткое содержание.  

In [ ]:
# функция, которая объединяет все выше сделанные действия 
def text_to_sentences(filename):
    f = open(filename, "r", errors = 'ignore') 
    a = f.read()
    text =[]
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
    for s in sentences:
        text.append(s)
    return text
    
def preprocess(filename):
    f = open(filename, "r", errors = 'ignore') 
    a = f.read()
    text =[]
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(a)])
    for s in sentences:
        text.append(s)
    res = list(sent_to_words(text))
    res_ = []
    s_ =[]
    for s in res:
        for word in s:
            if word not in stopwords_english:
                s_.append(word)
        res_.append(s_)
        s_ = []
    res_fin = []
    s_fin = []
    for s in res_:
        for word in s:
            s_fin.append(lemmatizer.lemmatize(word))
        res_fin.append(s_fin)
        s_fin = []
    return res_fin

In [ ]:
file = '/Users/alina/Downloads/AMICorpusXML-master/data/ami-summary/extractive/ES2002a.extsumm.txt'
text = text_to_sentences(file)
f = preprocess(file)
words = []
for i in range(len(f)):
    for j in range(len(f[i])):
        words.append(f[i][j])
all_words = list(set(words))
matrix = []
vector = [0] * len(all_words)
for sent in f:
    for w in sent:
        if w in stopwords_english:
            continue 
        vector[all_words.index(w)] += 1
    matrix.append(vector)
    vector = [0] * len(all_words)
matrix 

[[0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
from sklearn_extra.cluster import KMedoids
def summary_mead(file):
    text = text_to_sentences(file)
    f = preprocess(file)
    words = []
    for i in range(len(f)):
        for j in range(len(f[i])):
            words.append(f[i][j])
    all_words = list(set(words))
    matrix = []
    vector = [0] * len(all_words)
    for sent in f:
        for w in sent:
            if w in stopwords_english:
                continue 
            vector[all_words.index(w)] += 1
        matrix.append(vector)
        vector = [0] * len(all_words)
    kmedoids = KMedoids(n_clusters=2, random_state=42).fit(matrix)
    kmedoids_centers = kmedoids.cluster_centers_
    kmedoids_centers = list(kmedoids_centers)
    for i in range(len(kmedoids_centers)):
        kmedoids_centers[i] = list(kmedoids_centers[i])
    indexes_sum = []
    for i in kmedoids_centers:
        indexes_sum.append(matrix.index(i))
    final_summary = []
    for i in indexes_sum:
        final_summary.append(text[i])
    full_summary = '. '.join(final_summary)
    print('Summary: {}'.format(full_summary))

In [ ]:
summary_mead('/Users/alina/Downloads/AMICorpusXML-master/data/ami-transcripts/ES2002a.transcript.txt')

Summary: Uh. Uh


/Users/alina/opt/anaconda3/lib/python3.7/site-packages/sklearn_extra/cluster/_k_medoids.py:235: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  "its corresponding cluster ({k}).".format(k=k)


## Abstractive methods 

### Bert

In [4]:
test2 = open('/content/ES2002a.transcript.txt', "r", errors = 'ignore')
transcript_sum_abstractive = test2.read()

In [10]:
test2 = open('/content/ES2002a.abssumm.txt', "r", errors = 'ignore')
summary_abstractive = test2.read()

In [1]:
#!pip install bert-extractive-summarizer

     |████████████████████████████████| 2.1MB 5.9MB/s 
     |████████████████████████████████| 901kB 18.0MB/s 
     |████████████████████████████████| 3.3MB 28.4MB/s 


In [2]:
#!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.5MB/s 


In [3]:
from summarizer import TransformerSummarizer
from summarizer import Summarizer

In [5]:
bert_model = Summarizer()
bert_summary = ''.join(bert_model(transcript_sum_abstractive, min_length=60))
print(bert_summary)


Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them. Uh, right, well basically um high priority for any animal for me is that they be willing to take a lot of physical affection from their family. Probably when he was little he got lots of attention for doing it and has forever been conditioned. ' Like so sort of like how do you I I mean one one way of looking at it would be, well the people producing television sets, maybe they have to buy remote controls. Or another way is maybe people who have T_V_ sets are really fed up with their remote control and they really want a better one or something. Okay so Right, so in function one of the priorities might be to combine as many uses I think so. Well like um, maybe what we could use is a sort of like a example of a successful other piece technology is palm palm pilots. So, like, I wonder if we might add something new to the to the remote control market, such as the lighting in yo

In [12]:
rouge = Rouge()
scores = rouge.get_scores(bert_summary, summary_abstractive)
print('Bert: {} '.format(scores))

Bert: [{'rouge-1': {'f': 0.12195121704830464, 'p': 0.07122507122507123, 'r': 0.423728813559322}, 'rouge-2': {'f': 0.009803919129662275, 'p': 0.005714285714285714, 'r': 0.034482758620689655}, 'rouge-l': {'f': 0.10572686963845608, 'p': 0.0625, 'r': 0.34285714285714286}}] 


### GPT2

In [13]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full_gpt2 = ''.join(GPT2_model(transcript_sum_abstractive, min_length=60))
print(full_gpt2)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'h.12.attn.masked_bias', 'h.13.attn.masked_bias', 'h.14.attn.masked_bias', 'h.15.attn.masked_bias', 'h.16.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.19.attn.masked_bias', 'h.20.attn.masked_bias', 'h.21.attn.masked_bias', 'h.22.attn.masked_bias', 'h.23.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Then they're small cute and furry, and uh when planet of the apes becomes real, I'm gonna be up there with them. So cost like production cost is twelve fifty, but selling price is is that wholesale or retail? Um f frequencies or something um as well as uh characters, um different uh keypad styles and s symbols. Like, p personally for me, at home I've I've combined the um the audio video of my television set and my D_V_D_ player and my C_D_ player. And um, you know, when I think about what they are now, it's better, but actually it's still kind of, I dunno, like a massive junky thing on the table. Well, um are we at ma right now on the assumption that our television remote control may have features which go beyond the television? So uh you get to draw your favourite animal and sum up your favourite characteristics of it. Um and what do I like about him, um That's just to suggest that his tail wags. He only does it after he's had his dinner and um he'll just all of a sudden just get up 

In [14]:
rouge = Rouge()
scores = rouge.get_scores(full_gpt2, summary_abstractive)
print('GPT2: {} '.format(scores))

GPT2: [{'rouge-1': {'f': 0.10781670891551215, 'p': 0.0641025641025641, 'r': 0.3389830508474576}, 'rouge-2': {'f': 0.010840105751574153, 'p': 0.006430868167202572, 'r': 0.034482758620689655}, 'rouge-l': {'f': 0.0921658959119965, 'p': 0.054945054945054944, 'r': 0.2857142857142857}}] 


### XLNet

In [ ]:
model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
full = ''.join(model(body, min_length=60))
print(full)

### Hugging face transformer (BERT) 

Как работают трансформемы? (главная фишка состоит в том, что убираются связи rnn во времени) 
RNN принимает слова последовательно, мы подаем слово и ждем, пока rnn закончет его обрабатывать, а потом давать второе. Transformer избавляется от rnn. Все, что нужно:  
1. ReLu 
2. Batch normalization 
3. attention   
4. embedding layers  (первые слови, когда подоем сетки слова, переведенные в числа) 
5. residual connections  
6. positional encoding. 
7. softmax layer 

Encoder: 
(параллельно обрабатываем слова в предложении независимо друг от друга) 
embedding layer + positional encoding + 6 main blocks  

## Идеи для чего-то нового

? сделать логистическую регрессию с большим числом признаков, например, со сколькими предложениями имеется связь (по similarity matrix), вес ребра из LexRank algo, в какой кластер предложение определилось по алгоритму k-means, score предложения с помощью меры td-idf  
? может, брать в каждой модели среднее от всех резульаттов embeddings  
? попробовать привязать LDA  
? Есть такой алгоритм, как  ***Skip-thought vector***  
Будем использовать фреймворк кодера/декодера для генерации векторов признаков.  
Кодер: Кодировщик обычно представляет собой GRU-RNN, который генерирует векторное представление фиксированной длины h(i) для каждого предложения S(i) на входе. Кодированное представление h(i) получается путем передачи конечного скрытого состояния ячейки GRU.  
Декодер: Сеть декодера принимает это векторное представление h(i) в качестве входных данных и пытается сгенерировать два предложения — S(i-1) и S(i+1), которые могут возникнуть до и после входного предложения соответственно. Отдельные декодеры реализуются для генерации предыдущего и следующего предложений, оба являются GRU-RNN. Векторное представление h(i) действует как начальное скрытое состояние для GRU сетей декодера.  
Подобно тому, как вложения Word2Vec обучаются предсказанию окружающих слов, skip-thought vector обучается предсказанию окружающих предложений. По мере обучения этой модели изученное представление (скрытый слой) теперь будет размещать похожие предложения ближе друг к другу, что обеспечивает более семантически связную кластеризацию.
Можно будем применить extractive method, а затем для обеспечения баланса в предложений и отсуствия разрывных предложений, можно использовать skip-thought vector.
? Использовать вместо Word2Vec embedding Милокова

Нужно подумать, какая должна быть структура саммари делового совещания (сначала должна выдаваться дата совещания...) и как работать с диалогами. 

## Проблемы, с которыми можно будет побороться

1. Длинные предложения, решение: если предложение превышает определнную длину, то делать реферирования сначало в нем

## Распознование речи 

https://habr.com/ru/company/yandex/blog/337572/  
https://habr.com/ru/post/351376/

## Список литературы


[1]  H.P.LUHN "The Automatic Creation of Literature Abstracts"      https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf  
[2]  Dipanjan Das, Andre F.T. Martins "A Survey on Automatic Text Summarization"    
https://www.cs.cmu.edu/~nasmith/LS2/das-martins.07.pdf  
[3]  Edmundson H. P. (1969), "New methods in automatic extracting, Journal of the
ACM", vol. 16 Issue 2, pp. 264–285.  
[4]  DeJong G. F. (1982), An Overview of the FRUMP System, in Strategies for Natural Language Processing, Lawrence Erlbaum, Hillsdale, New Jersey, pp. 149–176.  
[5]  "Variations of the Similarity Function of TextRank for Automated Summarization"
Federico Barrios, Federico L´opez, Luis Argerich, Rosita Wachenchauzer. Facultad de Ingenier´ıa, Universidad de Buenos Aires, Ciudad Aut´onoma de Buenos Aires, Argentina. Universidad Nacional de Tres de Febrero, Caseros, Argentina.  
[6] K.NandhiniS.R.Balasundaram "Improving readability through extractive summarization for learners with reading difficulties"
[7] Dineshnath .G "Abstractive Text Summarization of Research Articles Based on Word Associations"  
[8] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin "Attention is all you need"  
[9] Rahul Katragadda, Prasad Pingali, Vasudeva Varma "Sentence Position revisited: A robust light-weight Update Summarization ‘baseline’ Algorithm"  
[10] Josef Steinberger, Karel Ježek "Using Latent Semantic Analysis in Text Summarization and Summary Evaluation"      
[11] Sandeep Sripadaa, Jagadeesh Jagarlamudi "Summarization Approaches Based on Document Probability Distributions"  
[12] Новосёлова Анастасия Максимовна "Исследование методов автоматического реферирования текстов"